# Dataset Sweep usign XGBoost on GPU
Trying different variations on the dataset using PCA, other techniques

# Setup

In [1]:
# two manual flags (ex-config)
colab = False
gpu_available = True

In [2]:
# basic imports
from pathlib import Path
import os
import math
from datetime import datetime
import random
import multiprocessing
import pickle

In [3]:
%matplotlib inline
%config Completer.use_jedi = False
os.environ['WANDB_NOTEBOOK_NAME'] = f"sweep_xgboost_{datetime.now().strftime('%Y%m%d')}.ipynb"

In [4]:
# handle Google Colab-specific library installation/updating
if colab:
    # much of the below inspired by or cribbed from the May 2021 Kaggle Tabular Playground winner, at 
    # https://colab.research.google.com/gist/academicsuspect/0aac7bd6e506f5f70295bfc9a3dc2250/tabular-may-baseline.ipynb?authuser=1#scrollTo=LJoVKJb5wN0L
    
    # Kaggle API for downloading the datasets
    !pip install --upgrade -q kaggle

    # weights and biases
    !pip install -qqqU wandb
    
    # Optuna for parameter search
    !pip install -q optuna

    # !pip install --upgrade xgboost

    # upgrade sklearn
    !pip install --upgrade scikit-learn

    # !pip install category_encoders
    # !pip install catboost
#     !pip install --upgrade -q lightgbm

    # lighgbm gpu compatible
    # !git clone --recursive https://github.com/Microsoft/LightGBM
    # ! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
    
    # # this part is from https://github.com/rapidsai/gputreeshap/issues/24
    # !pip install cmake --upgrade
    # # !pip install sklearn --upgrade
    # !git clone --recursive https://github.com/dmlc/xgboost
    # %cd /content/xgboost
    # !mkdir build
    # %cd build
    # !cmake .. -DUSE_CUDA=ON
    # !make -j4
    # %cd /content/xgboost/python-package
    # !python setup.py install --use-cuda --use-nccl
    # !/opt/bin/nvidia-smi
    # !pip install shap
    

Now, non-stdlib imports

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import log_loss, roc_auc_score
import wandb
from optuna.integration.wandb import WeightsAndBiasesCallback
# from wandb.xgboost import wandb_callback
# from wandb.lightgbm import wandb_callback
# from sklearn.impute import KNNImputer, StandardImputer
# import timm

import seaborn as sns

# from catboost import CatBoostClassifier
from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MaxAbsScaler, StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures
# from sklearn.feature_selection import SelectKBest, f_regression
from joblib import dump, load
# feature engineering tools
# from sklearn.feature_selection import mutual_info_regression
# import featuretools as ft


from optuna.samplers import TPESampler
import optuna
# import catboost
from sklearn.utils import resample
import sklearn.metrics

In [6]:
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression


Now, datapath setup

In [7]:
# # This is the code for reading the train.csv and converting it to a .feather file
# df = pd.read_csv(datapath/'train.csv', index_col='id', low_memory=False)
# df.index.name = None
# df.to_feather(path='./dataset_df.feather')

In [8]:
if colab:
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # handling datapath
#     datapath = Path('/content/drive/MyDrive/kaggle/tabular_playgrounds/sep2021/')
    
else:
    # if on local machine
#     datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/')  
    root = Path('/home/sf/code/kaggle/tabular_playgrounds/oct2021/')
    datapath = root/'datasets'
    edapath = root/'EDA'
    modelpath = root/'models'
    predpath = root/'preds'
    subpath = root/'submissions'
    studypath = root/'optuna_studies'
    
    for pth in [root, datapath, edapath, modelpath, predpath, subpath]:
        pth.mkdir(exist_ok=True)

In [9]:

# n_trials = int(1000)
SEED = 42

In [10]:
# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(SEED)

## Ex-Model Config

In [11]:
# meta-config for preprocessing and cross-validation, but NOT for model parameters
# in the sweep version, this includes both ex-model parameters and defaults for model parameters
exmodel_config = {
    # model config
    "library": 'xgboost',
#     "model": XGBClassifier,
#     "n_estimators": 100, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "test_size": 0.2,
#     "reg_lambda": None, 
#     "scaler": "sklearn.preprocessing.StandardScaler()", # TODO: experiment with others (but imputation may be slow)
#     "scale_b4_impute": False,
#     "imputer": "sklearn.impute.SimpleImputer(strategy='median', add_indicator=True)",
#     "knn_imputer_n_neighbors": None, # None if a different imputer is used
#     "feature_selector": SelectKBest,
#     "k_best": 80,
#     "feature_selection_scoring": f_regression,
    'random_state': SEED,
    'optuna': True,
#     'optuna_trials': 50,
#     'subsample': 1,
#     'cross_val_strategy': None, # None for holdout, or the relevant sklearn class
#     'kfolds': 1, # if 1, that means just doing holdout
#     'test_size': 0.2,
    # these are XGBoost default (my choice) params 
#     "tree_method": "auto", # set to 'gpu_hist' to try GPU if available
#     "booster": 'gbtree', # dart may be marginally better, but will opt for this quicker approach as a default
#     "n_estimators": 200, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "n_jobs": -1,
#     "verbosity": 1,
#     "subsample": 1,
#     'features_created': False,
#     'feature_creator': None,
}

wandb_kwargs = {
    # wandb config
    'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
    'project': '202110_Kaggle_tabular_playground',
    'tags': ['sweep'],
    'notes': "Sweep for preprocessing techniques on dataset",
    'config': exmodel_config,
}

## Data Setup

**TODO** Write some conditional logic here to automate it -- possibly as part of a sklearn.*pipeline

In [12]:
train_source = datapath/'train.feather'
df = pd.read_feather(path=train_source)
df.index.name = 'id'
y = df.target
features = [x for x in df.columns if x != 'target']
X = df[features]
# X.index.name = 'id'
# y.index.name = 'id'
# X = np.array(X_train)
# y = np.array(y_train)

# del df, X_train, y_train


# exmodel_config['feature_count'] = len(X.columns)
# exmodel_config['feature_count'] = X.shape[1]
# exmodel_config['instance_count'] = X.shape[0]

# exmodel_config['feature_generator'] = None
# exmodel_config['feature_generator'] = "Summary statistics"

# exmodel_config['train_source'] = str(train_source)
test_source = datapath/'test.feather'
# exmodel_config['test_source'] = str(test_source)
X_test = pd.read_feather(path=test_source)
X_test = X_test.iloc[:, 1:]
# X_test = np.array(X_test)

# Experiment setup

In [13]:
# wandb_kwargs = {
#     # wandb config:
#     'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
#     'project': '202109_Kaggle_tabular_playground',
#     'tags': ['sweep'],
#     'notes': "Sweep for CatBoost using Optuna",
#     'config': exmodel_config,
# }

In [14]:
X.shape

(1000000, 285)

In [15]:
X_test.shape

(500000, 285)

In [16]:
X_test.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284
0,0.178216,0.435617,0.010230,0.202074,0.390170,0.324221,0.221722,0.738894,0.582588,0.343770,...,1,0,0,0,0,0,1,1,1,0
1,0.181250,0.476455,0.022413,0.283146,0.598020,0.349508,0.283467,0.721575,0.268990,0.208373,...,0,0,0,0,0,0,0,0,0,0
2,0.159721,0.451202,0.259649,0.365274,0.594634,0.413502,0.249318,0.642339,0.411104,0.246891,...,0,0,0,0,0,0,1,0,0,0
3,0.182424,0.520976,0.095344,0.327742,0.741830,0.358711,0.270077,0.601662,0.297742,0.252829,...,0,0,0,0,0,1,1,0,0,0
4,0.229329,0.336513,0.023511,0.300913,0.668738,0.481586,0.545660,0.667849,0.546045,0.202731,...,0,0,0,0,1,0,0,1,0,0


In [18]:
from boruta import BorutaPy

In [19]:
from xgboost import XGBRegressor

In [23]:
feature_picker = XGBClassifier(
    verbosity = 1,
    tree_method = 'gpu_hist',
    booster = 'gbtree',
    random_state = SEED
)

In [24]:
boruta = BorutaPy(
    verbose=2,
    random_state=SEED,
    estimator=feature_picker,
    n_estimators='auto',
    max_iter=200,
)

In [25]:
# boruta.fit(np.array(X), np.array(y))
boruta = load('/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/alt_datasets/boruta_200iter.joblib')

/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:47:24] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Iteration: 	1 / 200
Confirmed: 	0
Tentative: 	285
Rejected: 	0
[15:48:52] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Iteration: 	2 / 200
Confirmed: 	0
Tentative: 	285
Rejected: 	0
[15:51:03] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Iteration: 	3 / 200
Confirmed: 	0
Tentative: 	285
Rejected: 	0
[15:53:13] WARNING: ../src/learner.cc:1095: Starting in

BorutaPy(estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                 colsample_bylevel=1, colsample_bynode=1,
                                 colsample_bytree=1, gamma=0, gpu_id=0,
                                 importance_type='gain',
                                 interaction_constraints='',
                                 learning_rate=0.300000012, max_delta_step=0,
                                 max_depth=6, min_child_weight=1, missing=nan,
                                 monotone_constraints='()', n_estimators=250,
                                 n_jobs=16, num_parallel_tree=1,
                                 random_state=1807573427, reg_alpha=0,
                                 reg_lambda=1, scale_pos_weight=1, subsample=1,
                                 tree_method='gpu_hist', validate_parameters=1,
                                 verbosity=1),
         max_iter=200, n_estimators='auto',
         random_state=RandomState(MT19937) at 0

In [26]:
green_zone = X.columns[boruta.support_].to_list()
blue_zone = X.columns[boruta.support_weak_].to_list()

In [27]:
feature_ranks = boruta.ranking_

In [29]:
X_filtered = boruta.transform(np.array(X))

In [30]:
dump(X_filtered, '/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/alt_datasets/X_boruta_200iter_filtered.joblib')

['/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/alt_datasets/X_boruta_200iter_filtered.joblib']

In [31]:
# dump(boruta, '/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/alt_datasets/boruta_200iter.joblib')

['/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/alt_datasets/boruta_200iter.joblib']

In [33]:
len(green_zone)

98

In [34]:
len(blue_zone)

11

In [35]:
X_filtered.shape

(1000000, 98)

In [36]:
feature_ranks

array([ 79,   1,   1,   1,   1,   1,   1,   1,   1,  19,  87,  17,   1,
         1,   1,  54,   1,   1,   1,   1,   1, 145,   1, 110,  94,  55,
         1,  29, 131,   1,  72, 103,  73,   2,  59,  29,  69, 132,  89,
        60,   1, 108,   1,   1,   1, 151,  61,  49,   1, 141,  96, 120,
         1,   1,  42,   2,   1,  36,   1, 161,  50,  25,   8,   1,   2,
         1,  81,  70, 138,   1,  16,   1,   1,   1,   1,   1,   2,   1,
         1,  20,  25,  38,   1,   8,  97,   1,   1,   2,  98,   2,   1,
       125,   1,   1,  58,   1,   1, 154,  14,   1,  90,  52, 157,   1,
        66, 105,  94,   1,   2,  85, 129, 129,   1,  11,   3,  62,  56,
        18,   2,   1, 127,  76, 115,  83, 115,   1, 138,   1,  25,   2,
         1,  36, 101,   5,   1, 150,   1, 117,   1,   1,   1,   3, 118,
         1,   5, 142,  32,  27,  86, 147,   1, 104,   1,  44,   1, 112,
         1,   1, 162,  43,  79,  46,   1,   1,  22, 112, 112,  96, 116,
         1, 103, 153, 108,   1,   1,  72, 140,  48,  82,   1, 12

In [49]:
type(X)

pandas.core.frame.DataFrame

In [37]:
best_xgboost_params = {
        'n_estimators': 3878,
        'max_depth': 4,
        'learning_rate': 0.024785857161974977,
        'reg_alpha': 26.867682044658245,
        'reg_lambda': 10.839759074147148,
        'subsample': 0.8208581489835881,
        'min_child_weight': 8.829122644339664,
        'colsample_bytree': 0.906420714280384,
        'gamma': 1.472322916021486
    }

# instantiate the model, with some parameters locked in, and experimnental ones passed via splat 
model = XGBClassifier(
    objective='binary:logistic',
    verbosity=1,
    tree_method='gpu_hist',
    booster='gbtree', # not bothering with dart for time reasons
    random_state=SEED,
    **best_xgboost_params
#         n_jobs=-1,
#         **params
)    

In [38]:
y_np = np.array(y)

In [39]:
del y

In [40]:
type(X_filtered), type(y_np)

(numpy.ndarray, numpy.ndarray)

In [41]:
X_train, X_valid, y_train, y_valid = train_test_split(X_filtered, y_np, test_size=0.2, random_state=int(SEED), shuffle=True)

model.fit(X_train, y_train)

/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:14:11] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.906420714280384,
              gamma=1.472322916021486, gpu_id=0, importance_type='gain',
              interaction_constraints='', learning_rate=0.024785857161974977,
              max_delta_step=0, max_depth=4, min_child_weight=8.829122644339664,
              missing=nan, monotone_constraints='()', n_estimators=3878,
              n_jobs=16, num_parallel_tree=1, random_state=42,
              reg_alpha=26.867682044658245, reg_lambda=10.839759074147148,
              scale_pos_weight=1, subsample=0.8208581489835881,
              tree_method='gpu_hist', validate_parameters=1, verbosity=1)

In [42]:
# generate predictions
preds = model.predict_proba(X_valid)[:,1]
# rounds to the nearest integer, and the nearest even in case of _.5s

# Evaluation
valid_auc = roc_auc_score(y_valid, preds)
print('ROC AUC Score of XGBoost =', valid_auc) # was 0.7783978025549229 for pca_poly; 0.8572984856383443 for vanilla

ROC AUC Score of XGBoost = 0.8553163413048461


So, it gets 0.8553163413048461, vs 0.8572984856383443 for the same model on the vanilla training set. Not bad.

Let's try a more conservative take -- the green zone *plus* the blue zone (i.e. the take-it-or-leave-it features).

In [43]:
del X_train, X_valid 

In [51]:
X_boruta_blue_green = np.concatenate((X_filtered, np.array(X[blue_zone])), axis=1)

In [52]:
X_boruta_blue_green.shape

(1000000, 109)

In [53]:
X_train, X_valid, y_train, y_valid = train_test_split(X_boruta_blue_green, y_np, test_size=0.2, random_state=int(SEED), shuffle=True)

model.fit(X_train, y_train)
# generate predictions
preds = model.predict_proba(X_valid)[:,1]
# rounds to the nearest integer, and the nearest even in case of _.5s

# Evaluation
valid_auc = roc_auc_score(y_valid, preds)
print('ROC AUC Score of XGBoost =', valid_auc) 

/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:52:35] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
ROC AUC Score of XGBoost = 0.8558487581638441


So that gets 0.8558487581638441 -- a bit better than the green-zone only. Now, let's try adding polynomial features.

In [ ]:
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_boruta_blue_green_poly = poly.fit_transform(X_boruta_blue_green)

In [ ]:
dump(X_boruta_blue_green, '/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/alt_datasets/X_boruta_200iter_green+blue_109features.joblib')
del X_boruta_blue_green

In [32]:
del X_pca_poly

In [33]:
train_source = datapath/'train.feather'
df = pd.read_feather(path=train_source)
df.index.name = 'id'
y = df.target
features = [x for x in df.columns if x != 'target']
X = df[features]
# X.index.name = 'id'
# y.index.name = 'id'
# X = np.array(X_train)
# y = np.array(y_train)

# del df, X_train, y_train


# exmodel_config['feature_count'] = len(X.columns)
# exmodel_config['feature_count'] = X.shape[1]
# exmodel_config['instance_count'] = X.shape[0]

# exmodel_config['feature_generator'] = None
# exmodel_config['feature_generator'] = "Summary statistics"

# exmodel_config['train_source'] = str(train_source)
test_source = datapath/'test.feather'
# exmodel_config['test_source'] = str(test_source)
X_test = pd.read_feather(path=test_source)
X_test = X_test.iloc[:, 1:]
# X_test = np.array(X_test)

In [34]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y_np, test_size=0.2, random_state=int(SEED), shuffle=True)

model.fit(X_train, y_train)

/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:48:25] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.906420714280384,
              gamma=1.472322916021486, gpu_id=0, importance_type='gain',
              interaction_constraints='', learning_rate=0.024785857161974977,
              max_delta_step=0, max_depth=4, min_child_weight=8.829122644339664,
              missing=nan, monotone_constraints='()', n_estimators=3878,
              n_jobs=16, num_parallel_tree=1, random_state=42,
              reg_alpha=26.867682044658245, reg_lambda=10.839759074147148,
              scale_pos_weight=1, subsample=0.8208581489835881,
              tree_method='gpu_hist', validate_parameters=1, verbosity=1)

In [35]:
# generate predictions
preds = model.predict_proba(X_valid)[:,1]
# rounds to the nearest integer, and the nearest even in case of _.5s

# Evaluation
valid_auc = roc_auc_score(y_valid, preds)
print('ROC AUC Score of XGBoost =', valid_auc)

ROC AUC Score of XGBoost = 0.8572984856383443


In [14]:
# originally from https://www.kaggle.com/satorushibata/optimize-catboost-hyperparameter-with-optuna-gpu
def objective(trial):
    # split the (original Kaggle training) data into partitions
    # if study.best_trial:
    #     print("Dumping best params, which are:")
    #     print(str(study.best_trial.params))
    #     dump(study.best_trial.params, filename=datapath/'optuna_catboost_best_20210920.joblib')
    
#     pca_components = trial.suggest_int('pca_components', 50, 285)
    pca = PCA(n_components=pca_components, random_state=42)
    X_pca = pca.fit_transform(X)
#     dump(pca60, edapath/'PCA_60.joblibg')
    
    # else:
    #     print("No best study yet")
    X_train, X_valid, y_train, y_valid = train_test_split(X_pca, y, test_size=0.33, random_state=int(SEED), shuffle=True)
    # create wrappers for the training and validation partitions
    # train_pool = catboost.Pool(X_train, y_train)
    # valid_pool = catboost.Pool(X_valid, y_valid)
    
    # experimental parameters
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 900, 6000), # was 900-4500 for CPU
#         'max_depth' : trial.suggest_int('depth', 3, 10),                                       
#         'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 0.4),               
#         'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.001, 30),
#         'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.001, 30),
#         'subsample': trial.suggest_uniform('subsample', 0.5, 1),
# #         'booster': trial.suggest_categorical('boosting_type', ['gbtree', 'dart']),
#         'min_child_weight': trial.suggest_uniform('min_child_weight', 0.001, 10),
#         'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1),
#         'gamma': trial.suggest_uniform('gamma', 0.1, 30)
#     }  

    best_xgboost_params = {
        'n_estimators': 3878,
        'max_depth': 4,
        'learning_rate': 0.024785857161974977,
        'reg_alpha': 26.867682044658245,
        'reg_lambda': 10.839759074147148,
        'subsample': 0.8208581489835881,
        'min_child_weight': 8.829122644339664,
        'colsample_bytree': 0.906420714280384,
        'gamma': 1.472322916021486
    }

    # instantiate the model, with some parameters locked in, and experimnental ones passed via splat 
    model = XGBClassifier(
        objective='binary:logistic',
        verbosity=1,
        tree_method='gpu_hist',
        booster='gbtree', # not bothering with dart for time reasons
        random_state=SEED,
        **best_xgboost_params
#         n_jobs=-1,
#         **params
    )    

    model.fit(X_train, y_train)
    # generate predictions
    preds = model.predict_proba(X_valid)[:,1]
    # rounds to the nearest integer, and the nearest even in case of _.5s

    # Evaluation
    valid_auc = roc_auc_score(y_valid, preds)
    print('ROC AUC Score of XGBoost =', valid_auc)
    wandb.log({'valid_auc': valid_auc,
              })

    return valid_auc

In [15]:
wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)

<ipython-input-15-69ea9289a2cf>:1: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find sweep_xgboost_20211010.ipynb
wandb: Currently logged in as: hushifang (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [16]:
study = optuna.create_study(direction = "maximize", 
                            sampler = TPESampler(seed=int(SEED)), 
                            study_name=f"pca_{datetime.now().strftime('%Y%m%d')}")

# study = load(studypath/f"optuna_xgboost_study_106trials_20211004.joblib")


[I 2021-10-10 11:57:06,723] A new study created in memory with name: pca_20211010


In [17]:
import torch

In [18]:
import xgboost

In [19]:
xgboost.core.XGBoostError?

Init signature: xgboost.core.XGBoostError(self, /, *args, **kwargs)
Docstring:      Error thrown by xgboost trainer.
File:           ~/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/core.py
Type:           type
Subclasses:     


In [20]:
for x in range(1,200):
    study.optimize(objective, n_trials = 1, callbacks = [wandbc], show_progress_bar=True, catch=(xgboost.core.XGBoostError,)) 
    dump(study, filename=datapath/f"optuna_dataset-pca_study_{x}trials_{datetime.now().strftime('%Y%m%d')}.joblib")
#     dump(study.best_trial.params, filename=datapath/f'optuna_lightgbm_study_best-thru-{x*5}trials_20210927.joblib')

/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/optuna/progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/1 [00:00<?, ?it/s]

/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:57:38] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


wandb: WARNING Step must only increase in log calls.  Step 0 < 1; dropping {'pca_components': 138, 'value': 0.8283967087232865}.


...

wandb: WARNING Step must only increase in log calls.  Step 14 < 15; dropping {'pca_components': 283, 'value': 0.8474194357944366}.


ROC AUC Score of XGBoost = 0.8474194357944366
[I 2021-10-10 12:39:09,826] Trial 14 finished with value: 0.8474194357944366 and parameters: {'pca_components': 283}. Best is trial 14 with value: 0.8474194357944366.


/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/optuna/progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# dump(study, filename=datapath/f"optuna_xgboost_100trials-complete_{datetime.now().strftime('%Y%m%d')}.joblib")
# dump(study.best_trial.params, filename=datapath/f"optuna_lightgbm_all-500trials-best_{datetime.now().strftime('%Y%m%d')}.joblib")
# pickle.dump(study.best_trial.params, open('CatBoost_Hyperparameter.pickle', 'wb'))
# print('CatBoost Hyperparameter:', study.best_trial.params)

In [ ]:
study.best_trial.params

In [ ]:
wandb.log({'xgboost_params': study.best_trial.params})
wandb.finish()

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)